In [8]:
import numpy as np
from linear_solvers import NumPyLinearSolver, HHL
from linear_solvers.matrices.tridiagonal_toeplitz import TridiagonalToeplitz
from scipy.sparse import diags
import qiskit
from qiskit import transpile, QuantumCircuit

In [9]:
qiskit.utils.algorithm_globals.massive=True

In [10]:
dimension = 5
a = 1
b = -1/3

In [11]:
# calculate the circuit depths for different number of qubits to compare the use
# of resources (WARNING: This will take a while to execute)
# matrix = diags([b, a, b],
#                [-1, 0, 1],
#                shape=(2**dimension, 2**dimension)).toarray()
vector = np.array([1] + [0]*(2**dimension -1))

# naive_hhl_solution = HHL().solve(matrix, vector)
# print('Naive Qubit count:', naive_hhl_solution.state.num_qubits)

tridi_matrix = TridiagonalToeplitz(dimension, a, b)
print("matrix state qubit size:", tridi_matrix.num_state_qubits)
tridi_solution = HHL().solve(tridi_matrix, vector)
print("Start transpiling...")
tridi_qc = transpile(tridi_solution.state,
                     basis_gates=['x', 'h', 'rz', 'add', 'cx'],
                     )
print("Total gates:", sum(tridi_qc.count_ops().values()))
print('CNOT count:', tridi_qc.count_ops()['cx'])
# Get OpenQASM files
qubits = tridi_qc.num_qubits
print('Qubit count:', qubits)
# Prevent printing of qasm: formatted=False
print('start writing qasm file...')
tridi_qc.qasm(formatted=False, filename="./raw_qasm/hhl_tri_n{}_from_python.qasm".format(qubits))
print("Done for dimension = {}".format(dimension))

matrix state qubit size: 5


: 

: 